# SIMULACIÓN KANEDA 

In [26]:
# https://manual.coppeliarobotics.com/en/remoteApiFunctionsPython.htm

In [27]:
# we import the necessary libraries
import numpy as np
import sympy as sp
from sympy import *
import math
import sim  


### CONEXIÓN CON COPPELIA

In [28]:
def connect(port):
    sim.simxFinish(-1) # just in case, close all opened connections
    clientID=sim.simxStart('172.21.0.1',port,True,True,2000,5) # Conectarse
    if clientID == 0: print("conectado a", port)
    else: print("no se pudo conectar")
    return clientID

In [223]:
clientID = connect(19999)

conectado a 19999


In [224]:
import time
retCode,wheel=sim.simxGetObjectHandle(clientID,'rueda_trasera',sim.simx_opmode_blocking)
print(wheel)






21


In [31]:
import torch
# acquisition of the Yolo5v Model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # o yolov5n - yolov5x6...

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-3-22 Python-3.11.7 torch-2.2.1+cu121 CPU

100%|██████████| 14.1M/14.1M [00:00<00:00, 29.0MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [117]:
import numpy as np              # maths
import cv2
from  matplotlib import pyplot as plt
clientID = connect(19999)

conectado a 19999


In [119]:
## Visión de la cámara
retCode,sensorHandle=sim.simxGetObjectHandle(clientID,'Vision_sensor',sim.simx_opmode_blocking)

retCode, resolution, image=sim.simxGetVisionSensorImage(clientID,sensorHandle,0,sim.simx_opmode_oneshot_wait)
img=np.array(image).astype(np.uint8)
img.resize([resolution[1],resolution[0],3])
img = img[::-1, ::1, :]
plt.imshow(img)

In [366]:
# Inference
results = model(img)

# Results
positions = results.pandas().xyxy[0]
positions["xcenter"] = [(positions["xmax"][i] - positions["xmin"][i])/2 + positions["xmin"][i] for i in range(len(positions))]
positions["ycenter"] = [(positions["ymax"][i] - positions["ymin"][i])/2 + positions["ymin"][i] for i in range(len(positions))]

# Change the centers from 0-1000 to 0-0.5 to use the map coordinates

coords = np.array([[positions["xcenter"][i]*0.5/1000, positions["ycenter"][i]*0.5/1000] for i in range(len(positions))])
positions["x"] = coords[:, 0]
positions["y"] = coords[:, 1]
positions
detections = positions.drop(columns=["xmin", "ymin", "xmax", "ymax", "confidence", "class", "xcenter", "ycenter"])
print(detections)
if not detections.empty:
    detection = detections.iloc[0]
    if round(detection.iloc[1], 2) == 0.11 and round(detection.iloc[2], 2) == 0.03:
        print("nos paramos") 



        name         x         y
0  stop sign  0.114012  0.031834
nos paramos


In [3]:
## IMPORTS INICIALES
import numpy as np
import time
import math
import sim  
import random as rand

## Recognition
import torch
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-3-22 Python-3.11.7 torch-2.2.1+cu121 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [9]:
## Funciones útiles
def connect(port):
    sim.simxFinish(-1) # just in case, close all opened connections
    clientID=sim.simxStart('172.21.0.1',port,True,True,2000,5) # Conectarse
    if clientID == 0: print("conectado a", port)
    else: print("no se pudo conectar")
    return clientID

def movimiento(target_velocity, joint):
    retCode = sim.simxSetJointTargetVelocity(clientID, joint, target_velocity, sim.simx_opmode_streaming)
    if retCode > 1: print("Movimiento Return:" + str(retCode) + " " + str(joint))

def mov_inercia(target_velocity, joint):
    retCode = sim.simxSetJointTargetVelocity(clientID, joint, target_velocity, sim.simx_opmode_streaming )
    if retCode > 1: print("Movimiento Return:" + str(retCode) + " " + str(joint))

def girar(joint, angle=(40*np.pi)/180):
    retCode= sim.simxSetJointTargetPosition(clientID,joint,angle,sim.simx_opmode_oneshot)
    if retCode > 1: print("giro Return:" + str(retCode))

def getImage(sensor):
    retCode, resolution, image=sim.simxGetVisionSensorImage(clientID,sensor,0,sim.simx_opmode_oneshot_wait)
    img=np.array(image).astype(np.uint8)
    img.resize([resolution[1],resolution[0],3])
    img = img[::-1, ::1, :]
    return img

def getDetections(img):
    # Inference
    results = model(img)

    # Results
    positions = results.pandas().xyxy[0]
    positions["xcenter"] = [(positions["xmax"][i] - positions["xmin"][i])/2 + positions["xmin"][i] for i in range(len(positions))]
    positions["ycenter"] = [(positions["ymax"][i] - positions["ymin"][i])/2 + positions["ymin"][i] for i in range(len(positions))]

    # Change the centers from 0-1000 to 0-0.5 to use the map coordinates
    coords = np.array([[positions["xcenter"][i]*0.5/1000, positions["ycenter"][i]*0.5/1000] for i in range(len(positions))])
    positions["x"] = coords[:, 0]
    positions["y"] = coords[:, 1]
    detections = positions.drop(columns=["xmin", "ymin", "xmax", "ymax", "confidence", "class", "xcenter", "ycenter"])
    #print(detections)
    return detections


In [12]:
## Connection
clientID = connect(19999)

## Handlers
retCode,sensorHandle=sim.simxGetObjectHandle(clientID,'Vision_sensor',sim.simx_opmode_blocking)
retCode,manillar=sim.simxGetObjectHandle(clientID,'manillar',sim.simx_opmode_blocking)
retCode,rueda_trasera=sim.simxGetObjectHandle(clientID,'rueda_trasera',sim.simx_opmode_blocking)
retCode,inercia=sim.simxGetObjectHandle(clientID,'MotorDC',sim.simx_opmode_blocking)
print(sensorHandle, manillar, rueda_trasera, inercia)

conectado a 19999
50 15 21 24


In [13]:
## Marcar movimiento inicial
movimiento(1, rueda_trasera) # avanzar
## BUCLE PRINCIPAL para detectar señal
flag = False
while not flag:
    mov_inercia(rand.randint(-20,20), inercia) # avanzar
    image = getImage(sensorHandle)
    detections = getDetections(image)
    if not detections.empty:
        detection = detections.iloc[0]
        if detection.iloc[0] == 'stop sign':
            if round(detection.iloc[1], 2) <= 0.10 and round(detection.iloc[2], 2) <= 0.034:
                flag = True
                movimiento(0, rueda_trasera) # frenar
                time.sleep(3)
## Giro
movimiento(1, rueda_trasera)
mov_inercia(20, inercia)
time.sleep(6)
girar(manillar)
mov_inercia(-20, inercia)
time.sleep(13)
girar(manillar, 0)
## fin

while True:
    mov_inercia(rand.randint(-20,20), inercia) # avanzar
    time.sleep(0.5)